# Scraping Reddit Data  

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/TannerGilbert/Tutorials/blob/master/Reddit%20Webscraping%20using%20PRAW/Reddit%20API.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/TannerGilbert/Tutorials/blob/master/Reddit%20Webscraping%20using%20PRAW/Reddit%20API.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

![](https://www.redditstatic.com/new-icon.png)  
Using the PRAW library, a wrapper for the Reddit API, everyone can easily scrape data from Reddit or even create a Reddit bot.

In [1]:
#!pip install praw


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import praw
import pandas as pd

Before it can be used to scrape data we need to authenticate ourselves. For this we need to create a Reddit instance and provide it with a client_id , client_secret and a user_agent . To create a Reddit application and get your id and secret you need to navigate to [this page](https://www.reddit.com/prefs/apps).

In [3]:
reddit = praw.Reddit(client_id='...',
                     client_secret='...',
                     user_agent='...')

We can get information or posts from a specifc subreddit using the reddit.subreddit method and passing it a subreddit name.

In [6]:
# get hot posts from all subreddits
hot_posts = reddit.subreddit('all').hot(limit=10)
for post in hot_posts:
    print(post.title)

Not this year
This is NOT going to end well:
me_irl
Trump (claimed height of 6’3) standing next to Vivek Ramaswamy (5’10)
These kids are so pure
Meanwhile in China
Sports Scholarship
American Rule
Well done 🤣
nobody wants to work anymore [oc]


In [4]:
# get 10 hot posts from the Pikabu subreddit
hot_posts = reddit.subreddit('Pikabu').hot(limit=10)

Now that we scraped 10 posts we can loop through them and print some information.

In [5]:
for post in hot_posts:
    print(post.title)

Военной ситуации мегатред
Но запах! […] Это был запах… победы!
Нюдсы на каждый день
Циник. Злодейское [слайдер]
Когда не в курсе последних новостей.
Бывает ¯\_(ツ)_/¯
Извините, на наши 40 см не было картинки
Румынские бандиты
Так началось восстание машин...
Это неправильные пчёлы (с)


In [7]:
# Specify the URL of the Reddit post you want to parse
post_url = 'https://www.reddit.com/r/Pikabu/comments/110gz4q/наггетсы_проверенный_рецепт/'

# Create a submission object for the post
submission = reddit.submission(url=post_url)

In [9]:
for top_level_comment in submission.comments:
    print(top_level_comment.body)

- 1 кг куриного филе 
- 150г. панировочных сухарей. 
- 150г кукурузных хлопьев.
- 2 ст.л. кукурузной муки. Либо обычной пшеничной.
- 1 ч.л. с горкой паприки
- 1 ч.л. с горкой сухого чеснока
- 3 ч.л. с горкой соли
- 3/4 ч.л. черного перца.
- 2 ст.л. молока или нежирных сливок
- 2 яйца
- сахар -2ч.л.
- растительное масло
++++++++++++++++++++++++++++
- Мясо минут за 20 забросить в морозилку, чтобы его легче было аккуратно нарезать.
- Хлопья измельчить в блендере пока они не станут размером примерно с фракцию ваших сухарей. 
- Все специи объединить вместе с солью и перемешать.
- В большой миске смешать сухари, хлопья, муку,сахар и половину пряностей с солью.
- В другой емкости смешать яйца, молоко и оставшуюся половину пряностей. Перемешать, но не взбивать.
- Нарезать куриное филе достаточно крупно.
- Забросить курицу в яичную смесь и хорошо перемешать, чтобы все было покрыто. Оставить минут на 5.
- Противень застелить пергаментом. 
- Каждый кусок курицы кидать в миску с панировочной смесь

In [28]:
# Choosing subreddit, number of posts to extract
subreddit_name = 'Pikabu'
num_posts_to_extract = 1000 

all_comments = []

# Iterating through the specified number of posts in the subreddit
for submission in reddit.subreddit(subreddit_name).new(limit=num_posts_to_extract):
    submission.comments.replace_more(limit=None)

    # Iterating through the comments and appending them to the all_comments list
    for comment in submission.comments.list():
        all_comments.append({
            'Author': str(comment.author),
            'Score': comment.score,
            'Comment': comment.body,
            'Submission Title': submission.title,
        })

# Export to a CSV file:
df = pd.DataFrame(all_comments)
df.to_csv('reddit_comments_pikabu.csv', index=False)

In [29]:
df.head()

,Author,Score,Comment,Submission Title
0,RECabu,2,Записал на видеокассету **[Вечное сияние чисто...,Будильник
1,SamSamABC1,3,может как это? - \n\nhttps://preview.redd.it/0...,"Выглядит залипательно, как какой-то физический..."
2,IvanovRomannn,-1,Напоминает нашу демократию,"Выглядит залипательно, как какой-то физический..."
3,RECabu,1,Записал на видеокассету **[Вспомнить всё](http...,"Выглядит залипательно, как какой-то физический..."
4,Initial-Carpenter,1,Красивое,"Выглядит залипательно, как какой-то физический..."


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20240 entries, 0 to 20239
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Author            20240 non-null  object
 1   Score             20240 non-null  int64 
 2   Comment           20240 non-null  object
 3   Submission Title  20240 non-null  object
dtypes: int64(1), object(3)
memory usage: 632.6+ KB
